# Load utils and store variables

In [11]:
%load_ext autoreload
%autoreload 2
import itertools
import os
import numpy as np
from collections import defaultdict
import operator
import pandas as pd
import matplotlib.pyplot as plt
import copy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Loading list of clustersb- from k=2 to 11  for 194 queries!!!
%store -r resultDictList
print(len(resultDictList))

10


In [3]:
# Load all combi
%store -r all_combi
print(len(all_combi))

46656


# Load all data per query

In [4]:
# Read all files with results - one per query
def loadQidBM25P(dirPath):
    os.chdir(dirPath)
    qidDict = dict()
    for filename in os.listdir(os.curdir):
        if filename.startswith("trec-eval-all-weights-qid-"):
            queryID = int(filename.split(".")[0].split("-")[-1])
            with open(filename) as f:
                weights2metrics = dict()
                for line in f:
                    if len(line.split("\t"))==2:
                        weight_combo, metrics = line.replace("\n", "").split("\t")
                        weights2metrics[weight_combo] = metrics
                    else:
                        print(line)
            qidDict[queryID] = weights2metrics
    return qidDict

In [5]:
qidDict = loadQidBM25P("/home/muntean/terrier-passage/tfs-per-qid")

In [6]:
print(len(qidDict))
print(qidDict.keys())
# print(len(qidDict[71]))
# print(qidDict[71]["111111"])
# print(qidDict[71]["111345"])

200
dict_keys([25, 103, 171, 15, 177, 18, 112, 62, 37, 198, 134, 68, 163, 193, 138, 29, 34, 164, 78, 128, 8, 89, 92, 61, 95, 197, 195, 153, 35, 140, 85, 57, 101, 47, 176, 5, 139, 31, 65, 99, 48, 50, 53, 82, 26, 90, 94, 187, 51, 137, 69, 148, 58, 106, 175, 91, 143, 196, 28, 86, 49, 2, 16, 64, 127, 132, 93, 54, 100, 125, 133, 179, 141, 189, 80, 130, 3, 191, 178, 17, 24, 122, 116, 39, 87, 20, 158, 46, 126, 108, 23, 167, 6, 73, 129, 102, 149, 14, 27, 88, 190, 155, 63, 160, 194, 135, 124, 71, 36, 168, 12, 174, 156, 119, 97, 19, 21, 180, 67, 104, 59, 43, 1, 84, 200, 184, 186, 146, 98, 52, 185, 13, 131, 60, 152, 162, 120, 118, 166, 42, 157, 154, 161, 105, 74, 199, 4, 7, 182, 165, 72, 70, 83, 30, 151, 145, 96, 77, 79, 40, 142, 45, 111, 38, 172, 150, 114, 76, 144, 170, 55, 136, 66, 32, 10, 115, 181, 173, 169, 9, 11, 123, 192, 44, 75, 109, 107, 159, 41, 117, 121, 56, 22, 33, 110, 81, 113, 183, 147, 188])


# Group queries intro cluster and select trec_eval metric

In [7]:
def getMetricPerQuery(weights2metrics, metric="recall_1000"):
    metricDict = dict()
    metricList = list()
    for weight, metric_str in weights2metrics.items():
        all_metrics = metric_str.split(",")
        for m in all_metrics:
            if m.startswith(metric):
                metricDict[int(weight)] = float(m.split(":")[2])
    return metricDict

In [8]:
def getClusterTable(resultDict, qidDict):
    cluster2table = defaultdict(list)
    for cluster, queries in resultDict.items():
        for qid, metric_str in qidDict.items():
            if qid in queries:
                dictQ = getMetricPerQuery(qidDict[qid], "recall_1000")
                cluster2table[cluster].append(dictQ)  
    return cluster2table

In [ ]:
# print(len(cluster2table))
# print(cluster2table.keys())
# print(len(cluster2table[1]))
# print(cluster2table[1][1].keys())
# print(len(cluster2table[1][1].keys()))

In [ ]:
cluster2tableK2 = getClusterTable(resultDictList[0], qidDict)

In [ ]:
cluster2tableK3 = getClusterTable(resultDictList[1], qidDict)

In [ ]:
resultDictK4 = copy.deepcopy(resultDictList[2])
del resultDictK4[3]
resultDictK4[1].append(150)

cluster2tableK4 = getClusterTable(resultDictK4, qidDict)

In [12]:
resultDictK5 = copy.deepcopy(resultDictList[3])
del resultDictK5[3]
resultDictK5[1].append(150)

cluster2tableK5 = getClusterTable(resultDictK5, qidDict)

In [ ]:
resultListAltered = [resultDictList[0], resultDictList[1], resultDictK4, resultDictK5 ]
%store resultListAltered

In [ ]:
cluster2best = defaultdict(list)
cluster2classic = defaultdict(list)
weightdictCluster = dict()
weightdictCluster[0] = 211112
weightdictCluster[1] = 211213
for cluster, queries in resultDictList[0].items():
    for qid, metric_str in qidDict.items():
        if qid in queries:
            dictQ = getMetricPerQuery(qidDict[qid], "recall_1000")
            cluster2best[cluster].append(dictQ[weightdictCluster[cluster]])  
            cluster2classic[cluster].append(dictQ[111111]) 

In [9]:
# print(cluster2best)
# print(cluster2classic)


# FOR K=2 ANY DIFFERENCE IS NOT STATISTICALLY SIGNIFICANT
# WE NEED AT LEAST 5 % DIFFENRENCE FOR 179 QUERIES!

print(np.mean(cluster2best[0]), np.std(cluster2best[0]), len(cluster2best[0]))
print(np.mean(cluster2classic[0]), np.std(cluster2classic[0]), len(cluster2classic[0]))
print()
print(np.mean(cluster2best[1]), np.std(cluster2best[1]), len(cluster2best[1]))
print(np.mean(cluster2classic[1]), np.std(cluster2classic[1]), len(cluster2classic[1]))

NameError: name 'cluster2best' is not defined

# Aggregate results per cluster and get Dataframe after filtering weights bigger or equal than 111111 - Classic BM25

In [ ]:
def dsum(dict_list):
    ret = defaultdict(float)
    denominator = len(dict_list)
    for d in dict_list:
        for k, v in d.items():
            ret[k] += v/denominator
    return dict(ret)

In [ ]:
def getDF(cluster_dict, biggerThan=111111):
    a = sorted(cluster_dict.items(), key=lambda x: x[1], reverse=True)
    df = pd.DataFrame(a, columns=['Weighting', 'Metric'])
    if biggerThan is not None:
        u = float(df.loc[df['Weighting'] == biggerThan]["Metric"])
        return df.loc[df["Metric"] >= u]
    else:
        return df

In [ ]:
def getCluster2df(cluster2table):
    cluster2df = dict()
    cluster2df_all = dict()
    index = range(len(all_combi))
    for k,v in cluster2table.items():
        cluster_dict = dsum(v)
        cluster2df[k] = getDF(cluster_dict)  
        cluster2df_all[k] = getDF(cluster_dict, None)
    return cluster2df

In [ ]:
# cluster2dfK2 = getCluster2df(cluster2tableK2)
# cluster2dfK3 = getCluster2df(cluster2tableK3)
# cluster2dfK4 = getCluster2df(cluster2tableK4)
cluster2dfK5 = getCluster2df(cluster2tableK5)

# %store cluster2df

In [ ]:
# %store cluster2dfK2
# %store cluster2dfK2
# %store cluster2dfK4
# %store cluster2dfK5

# Look at data

In [ ]:
averageBest = 0
averageClassic = 0
cluster2weight = dict()
cluster2classic = dict()

for cluster, df in cluster2dfK5.items():
    print("Cluster:" , cluster)
    print()
    print("Max BM25P value", df.head(1))
    print()
    print("B25 classic", df.loc[df['Weighting'] == 111111])
    print()
    print("Number of BM25P bigger than baseline: ", len(df.index))
    print("---------------------------")
    print()
    averageBest += float(df.head(1)['Metric'])
    averageClassic += float(df.loc[df['Weighting'] == 111111]['Metric'])
    cluster2weight[cluster] = (int(df.head(1)['Weighting']),float(df.head(1)['Metric']))
    cluster2classic[cluster] = (111111,float(df.loc[df['Weighting'] == 111111]['Metric']))

In [ ]:
print("Average BM25P Best on all clusters: ",averageBest/len(cluster2dfK5))
print("Average BM25P Classic on all clusters: ",averageClassic/len(cluster2dfK5))
print()
print(cluster2weight)
print()
print(cluster2classic)

## See progression with 0

In [ ]:
def plotWeightInc(best_w, df):
    values = []
    labels = []
    for i in range(1,7):
        current_w = best_w[:i] + "1"*(6-i)
        labels.append(current_w)
        values.append(float(df.loc[df['Weighting'] == int(current_w)]['Metric']) - float(df.loc[df['Weighting'] == int("111111")]['Metric']))
    labels.append(best_w)
    
    print(values)
    print(labels)
    plt.figure()
    plt.plot(values, "o-")
    for i, val in enumerate(values):
        plt.annotate(str(val), (i,val))
    plt.xticks(range(6), labels)
    plt.title(best_w)
    plt.show()

In [ ]:
for cluster, df in cluster2df_all.items():
    best_w = str(cluster2weight[cluster][0])
    plotWeightInc(best_w, df)

In [ ]:
# Select random weight from the available values! 

best_weights = [x for x,y  in cluster2weight.values()]
print(best_weights)
best_weights = [111111 if x==0 else x for x in best_weights]
print(best_weights)

random_mean_100 = 0
for i in range(100):
    vals = list()
    for cluster, df in cluster2df_all.items():
        weight = np.random.choice(best_weights)
        vals.append(float(df.loc[df['Weighting'] == int(weight)]['Metric']))
        #print(cluster, weight, float(df.loc[df['Weighting'] == int(weight)]['Metric']))
    random_mean_100 += np.mean(vals)
print()
print("Average BM25P Random on all clusters: ", random_mean_100/100)
print("Average BM25P Best on all clusters: ",averageBest/len(cluster2df))
print("Average BM25P Classic on all clusters: ",averageClassic/len(cluster2df))